In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pypsa

plt.style.use(["bmh", "../../matplotlibrc"])
sns.set_palette("colorblind")

import matplotlib as mpl

mpl.rcParams["figure.dpi"] = 100

COUNTRY = "DE"

In [ ]:
files = {
    "perfectly inelastic": f"../../results/20240506-70a/network_solved_LT-country+{COUNTRY}-inelastic+true.nc",
    "VOLL 2000 €/MWh": f"../../results/20240506-70a/network_solved_LT-country+{COUNTRY}-voll+true.nc",
    "PWL-elastic": f"../../results/20240506-70a/network_solved_LT-country+{COUNTRY}-elastic_pwl+default.nc",
}
ns = {k: pypsa.Network(v) for k, v in files.items()}

In [ ]:
fig, axs = plt.subplots(
    3,
    1,
    figsize=(7, 6.5),
    sharey=True,
    sharex=False,
    gridspec_kw={"wspace": 0.1, "hspace": 0.1},
)

norm = plt.Normalize(0, 500)
cmap = "YlGn"

for i, ax in enumerate(axs.flatten()):

    key = list(ns.keys())[i]
    n = ns[key]
    lmp = n.buses_t.marginal_price["electricity"]
    lmp = lmp.groupby([lmp.index.year, lmp.index.month]).mean().unstack()

    sns.heatmap(lmp.T, ax=ax, square=True, cbar=False, cmap=cmap, norm=norm)

    ax.set_title(key, loc="left", fontsize="medium")
    ax.set_ylabel("Month")

    if i == 3:
        ax.set_xlabel("Year")
    else:
        ax.set_xlabel("")

    plt.setp(ax.get_yticklabels(), rotation=0)

    xtick_labels = [1951] + [year for year in range(1955, 2020, 5)]
    xtick_positions = [lmp.index.get_loc(year) for year in xtick_labels]

    ax.set_xticks(xtick_positions)
    ax.set_xticklabels(xtick_labels)
    plt.setp(ax.get_xticklabels(), rotation=0, ha="left")


cbar_ax = fig.add_axes([0.92, 0.2, 0.03, 0.6])

fig.colorbar(
    plt.cm.ScalarMappable(norm=norm, cmap=cmap),
    cax=cbar_ax,
    orientation="vertical",
    fraction=0.1,
    # shrink=0.5,
    label="Mean Electricity Price [€/MWh]",
    extend="max",
)

fig.subplots_adjust(right=0.9)

# plt.tight_layout()


for ext in ["png", "pdf"]:
    plt.savefig(
        f"../../figures/figure-heatmaps-{COUNTRY}.{ext}", bbox_inches="tight", dpi=300
    )